# TODO
#### Klassen protecten
```
import AutoMLWrapper.automlwrapper as aw
aw.Configuration.Configuration
```

#### Erweiterung um Bibliotheken
-  @__library.setter
    - a) Global/GlobalConfig.yaml
        - lib-names
        - type mappings
    - b) directory durchsuchen 
- import in AutoMLWrapper anpassen
- SEDAR automl.py config locations anpassen

### custom proprocessing
- typen checken
- die wrapper könnten property methods implementieren, die die Rückgabe eines festen samples prüfen

### custom parameters


### 2+ problem types

In [1]:
import io
import mlflow
import pandas as pd
import numpy as np
from PIL import Image

from AutoMLWrapper.tests import DataSamples

#from automlwrapper import AutoMLWrapper
from AutoMLWrapper.automlwrapper import AutoMLWrapper, SedarDataLoader



2024-01-29 20:14:53.985203: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 20:14:54.045253: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-29 20:14:54.046303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 20:14:55.055496: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend


In [2]:
RUN_ALL = 0
UPLOAD_ALL = 0

RUN_KERAS = 0
UPLOAD_KERAS = 0

RUN_SKLEARN = 0
UPLOAD_SKLEARN = 0

RUN_GLUON = 1
UPLOAD_GLUON = 0

user_hp = {'epochs':20, 'time_limit': 360, 'memory_limit':7000, 'testtest': 123, 'num_trials': 4,}

In [3]:
MLFLOW = 0

if MLFLOW:
    
    # #experiment_id = mlflow.create_experiment('test_automl', artifact_location='/home/mibbels/sedar-masterarbeit/sedar/sedar/automl/tmp/artifacts')
    
    experiment_id = 5
    remote_server_uri = "http://127.0.0.1:6798"
    mlflow.set_tracking_uri(remote_server_uri) 
    mlflow.set_experiment(experiment_id=experiment_id)

# Daten

In [4]:
""" Bilddaten """
#mnist_byte_df = DataSamples.create_mnist_bytearray_df(label_col='label', n_samples=-1)
#shopee_path_df = DataSamples.create_shopee_df(is_bytearray = False, n_samples=-1)
#mnist_tp = DataSamples.create_mnist_tuple(n_samples=-1)
#leaf_df = DataSamples.create_leaf_df(n_samples=-1)

#coco_df = DataSamples.create_coco_motorbike_df(n_samples=-1)

""" Textdaten """
#sentiment_df = DataSamples.create_sentiment_treebank_df(n_samples=-1)
#mloc_df = DataSamples.create_mloc_df(n_samples=-1)

""" Tabellendaten """
glass_df = DataSamples.create_glass_df()

""" Timeseries """
#m4_df = DataSamples.create_m4_df(n_samples=-1)

' Timeseries '

In [5]:
ask = AutoMLWrapper('autosklearn')
ask.SetOutputDirectory('test/ask/2')

if RUN_ALL or RUN_SKLEARN:
    ask.Train(
        data=glass_df,
        target_column='Type',
        task_type='classification',
        data_type='tabular',
        problem_type='multiclass',
        hyperparameters=user_hp
    )


if UPLOAD_ALL or UPLOAD_SKLEARN:
    print(ask.Output())
    ask.MlflowUploadBest({})

In [8]:
agl = AutoMLWrapper('autogluon')
agl.SetOutputDirectory('test/agl/9')


if RUN_ALL or RUN_GLUON: 
    train, test = SedarDataLoader.split_train_test(glass_df)
    agl.Train(
        data=train,
        target_column='Type',
        data_type='tabular',
        task_type='classification',        
        problem_type='multiclass',
        hyperparameters=user_hp
    )
    ev = agl.Evaluate(test)
    
if UPLOAD_ALL or UPLOAD_GLUON:
    print(agl.Output())
    agl.MlflowUploadBest({})

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 360s
AutoGluon will save models to "test/agl/9"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PVE 5.15.102-1 (2023-03-14T13:48Z)
CPU Count:          8
Memory Avail:       40.68 GB / 62.50 GB (65.1%)
Disk Space Avail:   113.32 GB

In [7]:
print(ev)


None


In [ ]:
akr = AutoMLWrapper('autokeras')
akr.SetOutputDirectory('test/akr/1')

if RUN_ALL or RUN_KERAS:
    akr.Train(data=mnist_tp,
        target_column='Type',
        task_type='classification',
        data_type='image',
        problem_type='multiclass',
        hyperparameters=user_hp
    )

if UPLOAD_ALL or UPLOAD_KERAS:
    print(akr.Output())
    akr.MlflowUploadBest({})

# MLFlow
~/miniconda3/envs/automl_env/bin/pip

In [ ]:
if MLFLOW:
    df = df_glass.drop(columns=['Type'])
    logged_model = 'runs:/a0337b9ecfe84e99b59ff3a9639f1ef6/model'
    loaded_model = mlflow.pyfunc.load_model(logged_model)

    #loaded_model.predict(test_data_byte)

# SEDAR API

# DataLoader TODO
- sedar api raw sourcedata
- sedar api get_all_images (image metadata)

- query_sourcedata nur structured
    - image
    - unstructured
    - semistructured

- keras format

# Ingestion
- metadaten?

In [ ]:
from sedarapi import SedarAPI
from AutoMLWrapper.automlwrapper.SedarDataLoader import SedarDataLoader

In [1]:
tabcl = ["13b4787c3e454649aa05a4cd680edc37", "986f2e837ca44f3e8c0ee7d2dc0c4287",
          "./data/sedar_raw/zip", "tabular", "classification", "Type"]
imseg = ["13b4787c3e454649aa05a4cd680edc37", "324ea420125d4167a76151b62368c4ad",
          "./data/sedar_raw/zip", "image", "segmentation", None]
imcl = ["13b4787c3e454649aa05a4cd680edc37", "513c6b1ee46b478c8e0925a098d2f387",
          "./data/sedar_raw/zip", "image", "classification", None]

from AutoMLWrapper.automlwrapper.SedarDataLoader import SedarDataLoader
from sedarapi import SedarAPI


def test_sedar_caafe(workspace_id, dataset_id, file_save_location, data_type, problem_type, target):
    base_url = "http://192.168.220.107:5000"
    email = "admin"
    password = "admin"
    sedar = SedarAPI(base_url)
    sedar.connection.logger.setLevel("INFO")
    sedar.login(email, password)

    DataLoader = SedarDataLoader(sedar)
    DataLoader.CAAFEFeatureEngineering(
        workspace_id, 
        dataset_id, 
        file_save_location,
        data_type,
        problem_type,
        target,
        dataset_description='No description',
        iterations=1
    )
    
test_sedar_caafe(*imseg)

2024-01-24 22:22:20.071766: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-24 22:22:20.129559: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-24 22:22:20.132147: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 22:22:22.463213: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend


INFO:SedarAPI-Logger:Login successful


INFO: the data has been written to: ./data/sedar_raw/zip/gitterschnitte_train.zip
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[ True  True  True ...  True  True  True]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]
[[0 0 0 ... 1 1 1]
 [1 0 0 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 0 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [1 1 0 ... 1 1 1]]
[[False False False ...  True 

2024-01-24 22:24:08.402060: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: assertion failed: [`labels` out of bound] [Condition x < y did not hold element-wise:] [x (confusion_matrix/control_dependency:0) = ] [255 255 255...] [y (confusion_matrix/Cast_2:0) = ] [2]
	 [[{{function_node confusion_matrix_assert_less_Assert_AssertGuard_false_2505}}{{node confusion_matrix/assert_less/Assert/AssertGuard/Assert}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'confusion_matrix/assert_less/Assert/AssertGuard/Assert' defined at (most recent call last):
    File "/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/site-packages/tornado/plat

In [ ]:
base_url = "http://192.168.220.107:5000"
email = "admin"
password = "admin"

sedar = SedarAPI(base_url)
sedar.connection.logger.setLevel("INFO")
sedar.login(email, password)

In [ ]:
workspace_id = '13b4787c3e454649aa05a4cd680edc37'
dataset_id = '324ea420125d4167a76151b62368c4ad'

# 986f2e837ca44f3e8c0ee7d2dc0c4287  - glass
# b5b74391e41e4634a54d5cffa059663b - coco_22_train_v1 
# 324ea420125d4167a76151b62368c4ad - gitter_train_v1
# 513c6b1ee46b478c8e0925a098d2f387  - test classification 1 img each class

DataLoader = SedarDataLoader(sedar)
#loc = DataLoader.query_data(workspace_id, dataset_id, file_save_location='./data/sedar_raw/test')
#X, y, mapping = DataLoader.zip_to_class_np_keras(loc[0], './data/sedar_raw/test/test_coco')


In [ ]:
from tabpfn.scripts import tabular_metrics

if 0:
    DataLoader.CAAFEFeatureEngineering(
        workspace_id, 
        '986f2e837ca44f3e8c0ee7d2dc0c4287', 
        file_save_location='./data/sedar_raw/test',
        data_type='tabular',
        problem_type='classification',
        target='Type',
        dataset_description='No description',
        iterations=1
    )

In [ ]:
DataLoader.CAAFEFeatureEngineering(
    workspace_id, 
    '324ea420125d4167a76151b62368c4ad', 
    file_save_location='./data/sedar_raw/test',
    data_type='image',
    problem_type='segmentation',
    dataset_description='No description',
)

In [ ]:
X, y = DataLoader.zip_to_segmentation_np_keras(loc[0], './data/sedar_raw/test/test_seg')

In [ ]:
from CAAFE.caafe import CAAFEImageClassifier, CAAFEImageSegmentor
from sklearn.metrics import accuracy_score
import openai



In [ ]:
if 1:
    import tensorflow as tf
    import numpy as np
    import keras

    cifar10 = keras.datasets.cifar10
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train, X_test = X_train / 255.0, X_test / 255.0

    # If you want to use the entire dataset as X and y
    X = np.concatenate((X_train, X_test), axis=0, dtype=np.float32)
    y = np.concatenate((y_train, y_test), axis=0, dtype=np.float32)

    #X = X.reshape(X.shape[0], 28, 28, 1)
    X = X[:1000]
    y = y[:1000]


In [ ]:
#model = "gpt-3.5-turbo"
model = "gpt-4"
caafe_clf = CAAFEImageClassifier(llm_model=model,
                                 iterations=3,
                                 )

pred = caafe_clf.performance_before_run(X, y)
acc = accuracy_score(pred, y)
print(f'Accuracy before CAAFE {acc}')


caafe_clf.fit_images(
    X,
    y,
    dataset_description="This is the CIFAR10 dataset."
    )

pred = caafe_clf.predict(X)
acc = accuracy_score(pred, y)
print(f'Accuracy after CAAFE {acc}')

X, y = caafe_clf.apply_code(X, y)

In [ ]:
_, y = caafe_clf.apply_code(X, y)
pred = caafe_clf.predict(X)
acc = accuracy_score(pred, y)
print(f'Accuracy after CAAFE {acc}')



In [ ]:
if 0:
    ws = sedar.get_workspace('13b4787c3e454649aa05a4cd680edc37')
    dataset = ws.get_dataset('986f2e837ca44f3e8c0ee7d2dc0c4287')
    
    # try:
    #     success = dataset.delete()
    #     if success:
    #         print("Dataset deleted successfully.")
    # except Exception as e:
    #     print(e)


In [ ]:
!cd ./AutoMLWrapper/tests && python3 -m unittest TestAutoMLWrapper.py